# 1. Identify the Companies of interest
-------------------
Group 3 , September 24, 2022
1. Gezhi Cheng, 
2. Haowei Lee, 
3. Ziyi Liu, 
4. VS Chaitanya Madduri

> <i>Description: This notebook understand and extract the list of the companies in the 2021 zip file. </i>


<div class="alert alert-block alert-info">
    <b>Tip:</b> #Please download and extarct the 2020 quarter1 files. 
</div> 

### Pre requisites: 
1. And add the shortcut of the drive link : https://drive.google.com/drive/folders/1tZP9A0hrAj8ptNP3VE9weYZ3WDn9jHic to your personal drive.

The above drive contains the 10-k fillings from the author 
Bill McDonald

Do to the huge files we have used our personal google drive folders to save the files.

Files:
No files 

### Output files:
1. And add the shortcut of the drive link :https://drive.google.com/drive/folders/1X4UdGsQiHVWSr63FRiz8rwOuWW5Ua8uI?usp=sharing to your personal drive.

Files:
Company_Details_analysis.csv - list of the companies selected for further analysis




## 1. Import Required Packages 

In [3]:
import os
import pandas as pd

In [179]:
class CompanyPreprocess:
    
    def __init__(self):
        self.companies = []
        self.industries = []
        self.company_inedx = []
        self.columns = ['Company_Name', 'Std_Indust_Class','Index_Key']
        self.master_dataframe = pd.DataFrame(columns= self.columns)
    
    @staticmethod
    def get_files_in_dir(dir_path):
        # list to store files
        files = []

        # Iterate directory
        for path in os.listdir(dir_path):
            # check if current path is a file
            if os.path.isfile(os.path.join(dir_path, path)):
                files.append(path)

        print(len(files))
        return files
    
    @staticmethod
    def get_first_few_files(files, num=100):
        '''
        get the some files in the zip folder based on the num.
        input:
        files: list of the files
        num: the count of the file
        
        return:
        selected number of the file
        '''
        first_few_files = files[:num]
        
        return first_few_files
    
    def extarct_company_details(self, file_data):
        '''
        The function extracts the key information from the file.
        file_data: the extracted data of the file as a python object
        return: (empty)
        '''
        
        index_val = self.master_dataframe.shape[0]
        for line in file_data:
            
            stripped_line = line.strip()
            if "COMPANY CONFORMED NAME:" in stripped_line :                
                splitted_str = stripped_line.split(":")  # split COMPANY CONFORMED NAME and real name
                company_name = splitted_str[1]        
                self.master_dataframe.loc[index_val,"Company_Name"] = company_name.strip()
                continue
                              

            if "CENTRAL INDEX KEY:" in stripped_line:
            
                splitted_str = stripped_line.split(":")  # split COMPANY CONFORMED NAME and real name
                company_inedx = splitted_str[1]
                self.master_dataframe.loc[index_val,"Index_Key"] = company_inedx.strip()
                continue
            
            if "STANDARD INDUSTRIAL CLASSIFICATION" in stripped_line:            
                splitted_str = stripped_line.split(":")  # split COMPANY CONFORMED NAME and real name
                industry_name = splitted_str[1]
                self.master_dataframe.loc[index_val,"Std_Indust_Class"] = industry_name.strip()
            
                break  
                
                
    def master_dataframe(self, list_data):
        '''
        Print the master dataframe to safe the files
        '''
        return self.master_dataframe
    
    def extract_file_data(self, file_path):        
        '''
        functions extracts the files 
        Input:
        file_path: (str) file path of the txt file 
        
        Return : (list) The text file split into a list
        '''
        with open(file_path) as f:
            return f.read().upper().split('\n')
    
    def get_company_data(self, file_path):
        '''
        Extracts the company details
        Input:
        file_path: (str) file path of the txt file 
        return: (empty)
        '''
        file_data = self.extract_file_data(file_path)
        self.extarct_company_details(file_data)
        
    
        

## 2. Extracting the company details from the QTR1

In [201]:
cp = CompanyPreprocess()

# kindly change the file path as per folder struture you have 
qtr1_file_path = r"C:\Users\chait\OneDrive - fs-students.de\Strategy\QTR1\\"


In [202]:
# Below command will extract the files in the QTR1 folder 
files_QTR1 = CompanyPreprocess.get_files_in_dir(dir_path)

# Below command will extract all the files in the QTR1 folder
first_few_files = CompanyPreprocess.get_first_few_files(files_QTR1, len(files_QTR1))

3314


In [203]:
# looping all the text files and extract the company details
# Note we are not retruning and data as we are updating the master dataframe directly
for file in first_few_files:
    cp.get_company_data(qtr1_file_path + file)

## 3. Validate the list and export to a csv

In [204]:
#Taking the copy of the master dataframe 
df = cp.master_dataframe

In [6]:
df.shape

(3314, 4)

In [15]:
# getting the industry code for the Std_Indust_Class
df['Class_number'] = df['Std_Indust_Class'].str[-5:-1]

In [24]:
# Extracting the firts 2 codes to group the companies with near by industry
df['Class_number_first'] = df['Class_number'].str[:2]

In [25]:
df.head()

,Unnamed: 0,Company_Name,Std_Indust_Class,Index_Key,Class_number,Class_number_first
0,0,ACCESS-POWER INC,TELEPHONE COMMUNICATIONS (NO RADIO TELEPHONE) ...,1041588,4813,48
1,1,"LUXXO, INC.",SERVICES-MANAGEMENT CONSULTING SERVICES [8742],1693801,8742,87
2,2,BIGFOOT PROJECT INVESTMENTS INC,SERVICES-MOTION PICTURE & VIDEO TAPE DISTRIBUT...,1569568,7822,78
3,3,HUBILU VENTURE CORP,SERVICES-MANAGEMENT CONSULTING SERVICES [8742],1639068,8742,87
4,4,CONAGRA BRANDS INC.,FOOD & KINDRED PRODUCTS [2000],23217,2000,20


In [32]:
# filtering the industry code with 49 and dropping the duplicates to get the unique company names 
df2 = df[df['Class_number_first']=='49'].drop_duplicates(['Company_Name'])

In [36]:
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [37]:
# Export the company details into the csv
df2.to_csv("Company_Details_analysis.csv", index=None)

### End of the Notebook